# Hackaton UBU 2025
## Problema de navegación marítima - Random Forest
**Autor**: [José Gallardo Caballero](mailto:jgc1031@alu.ubu.es)

<table>
    <tr>
        <td align="center"><a href="https://joseleelportfolio.vercel.app"><img src="https://github.com/Joseleelsuper.png" width="100px;" alt=""/><br /><sub><b>José Gallardo</b></sub></a></td>
    </tr>
</table>

En este ejercicio se debía de predecir, dados unos ejercicios  no etiquetados, si su dificultad individual y global era baja, media o alta. Para ello, se nos proporcionaban unos ejercicios que sí que estaban etiquetados con muestras de ejemplo.

Este es un típico problema de machine learning de clasificación, donde se utilizan técnicas de aprendizaje supervisado para predecir la clase a la que pertenece un nuevo dato en función de los datos de entrenamiento.

En mi caso, he utilizado un modelo de Random Forest, que es un algoritmo de aprendizaje automático basado en árboles de decisión. Este modelo es capaz de manejar datos no lineales y es robusto frente al sobreajuste, además, me permitió elegir unas cuantas FEATURES (Columnas) numéricas para hacer una media y permitir así al modelo predecir de la mejor forma posible el resultado del ejercicio.

In [1]:
# Librerías necesarias
import os
import glob
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import mode
import time

In [2]:
# Rutas
DATA_DIR = "./datos_etiquetados"
UNLABELED_DIR = "./datos_no_etiquetados"
OUTPUT_DIR = "./"

# Mapas de dificultad convertidos a números para el modelo
DIFICULTAD_MAP = {'BAJA': 0, 'MEDIA': 1, 'ALTA': 2}
DIFICULTAD_MAP_INV = {0: 'BAJA', 1: 'MEDIA', 2: 'ALTA'}

# Otros parámetros modificables
## Las features que se utilizarán en el modelo. Todas son las columnas de datos numéricos
FEATURES = ['DistanciaInicioRiiesgo', 'DCPA_yds', 'VelNudos', 'VelNudosContacto', 'TCPA']
## Nombre del CSV
CSV_NAME = 'EquipoReturn_predicciones.csv'

In [3]:
def load_situaciones_labeled(data_dir: str = DATA_DIR) -> pd.DataFrame:
    """Carga los datos etiquetados desde archivos CSV.

    Args:
        data_dir (str): Directorio donde se encuentran los datos etiquetados. Defaults to DATA_DIR.

    Returns:
        pd.DataFrame: DataFrame con los datos etiquetados.
    """
    all_files = glob.glob(os.path.join(data_dir, 'ejercicio_*.csv'))
    dfs = []
    for f in all_files:
        df = pd.read_csv(f, comment='#')
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    # Convertir etiquetas a números
    df = df[df['DificultadSituacion'].isin(DIFICULTAD_MAP)]
    df['DificultadSituacionNum'] = df['DificultadSituacion'].map(DIFICULTAD_MAP)
    return df

In [4]:
def train_situacion_model(df: pd.DataFrame) -> tuple:
    """Entrena un modelo de Random Forest para predecir la dificultad de una situación.
    Random Forest es un algoritmo de aprendizaje automático que utiliza múltiples árboles de decisión para mejorar la precisión y evitar el sobreajuste.
    Decidí utilizar este algoritmo porque es robusto y funciona bien con datos tabulares, además de que permite manejar las features numéricas de manera eficiente.

    Args:
        df (pd.DataFrame): DataFrame con los datos etiquetados.

    Returns:
        tuple: Una tupla con el modelo entrenado y los features utilizados para la predicción.
    """
    features = FEATURES
    X = df[features]
    y = df['DificultadSituacionNum']
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X, y)
    return clf, features

In [5]:
def predict_and_save_unlabeled(model: RandomForestClassifier, features: list, unlabeled_dir: str = UNLABELED_DIR, output_dir: str = OUTPUT_DIR):
    """Predice la dificultad de situaciones en archivos no etiquetados y guarda los resultados.

    Args:
        model (RandomForestClassifier): Modelo entrenado para predecir la dificultad.
        features (list): Lista de características utilizadas para la predicción.
        unlabeled_dir (str, optional): Directorio donde se encuentran los archivos no etiquetados. Defaults to UNLABELED_DIR.
        output_dir (str, optional): Directorio de salida. Defaults to OUTPUT_DIR.
    """
    all_files = glob.glob(os.path.join(unlabeled_dir, 'ejercicio_*.csv'))
    all_predictions = []
    
    for f in all_files:
        df = pd.read_csv(f, comment='#')
        # Predecir dificultad de cada situación
        X_pred = df[features]
        y_pred = model.predict(X_pred)
        df['DificultadSituacion'] = [DIFICULTAD_MAP_INV[v] for v in y_pred]
        # Dificultad global: moda de las situaciones
        dificultad_global = mode(y_pred, keepdims=True)[0][0]
        df['DificultadEjercicio'] = DIFICULTAD_MAP_INV[dificultad_global]
        
        # Añadir a la lista para el archivo combinado
        all_predictions.append(df)
    
    # Combinar todos los DataFrames y guardar en un solo CSV
    if all_predictions:
        combined_df = pd.concat(all_predictions, ignore_index=True)
        combined_path = os.path.join(output_dir, CSV_NAME)
        combined_df.to_csv(combined_path, index=False)
        print(f'Archivo combinado guardado en {combined_path}')

In [6]:
def summarize_data(data_dir: str = DATA_DIR):
    """Muestra estadísticas descriptivas y porcentajes de uso de dificultades."""
    # Datos etiquetados
    df = load_situaciones_labeled(data_dir)
    print("\nEstadísticas de características numéricas:")
    print(df[FEATURES].describe().loc[['mean', 'std', 'min', 'max']])
    print("\nPorcentaje de dificultades (situación):")
    print((df['DificultadSituacion'].value_counts(normalize=True) * 100).round(2))

    # Predicciones unlabelled
    pred_path = os.path.join(OUTPUT_DIR, CSV_NAME)
    if os.path.exists(pred_path):
        df_pred = pd.read_csv(pred_path)
        print("\nPorcentaje de dificultades por ejercicio (predicción):")
        print((df_pred['DificultadEjercicio'].value_counts(normalize=True) * 100).round(2))
    else:
        print("\nNo se encontró el archivo de predicciones. Ejecuta la función de predicción primero.")

def main():
    start_time = time.time()
    df = load_situaciones_labeled(DATA_DIR)
    print(f'Situaciones cargadas: {len(df)}')
    model, features = train_situacion_model(df)
    predict_and_save_unlabeled(model, features, UNLABELED_DIR, OUTPUT_DIR)
    print(f"Tiempo de ejecución: {time.time() - start_time:.2f} segundos")
    summarize_data(DATA_DIR)

if __name__ == "__main__":
    main()

Situaciones cargadas: 1150
Archivo combinado guardado en ./EquipoReturn_predicciones.csv
Tiempo de ejecución: 0.63 segundos

Estadísticas de características numéricas:
      DistanciaInicioRiiesgo     DCPA_yds   VelNudos  VelNudosContacto  \
mean             3143.337643   871.867400  13.658626          9.758870   
std              2566.897441   572.703934   4.934444          7.812196   
min               201.190000     1.000000   1.000000          0.000000   
max              9988.070000  1998.160000  26.600000         34.980000   

            TCPA  
mean    9.745029  
std    29.307722  
min     0.002340  
max   797.836792  

Porcentaje de dificultades (situación):
DificultadSituacion
MEDIA    47.48
ALTA     34.96
BAJA     17.57
Name: proportion, dtype: float64

Porcentaje de dificultades por ejercicio (predicción):
DificultadEjercicio
MEDIA    77.59
ALTA     14.07
BAJA      8.33
Name: proportion, dtype: float64


## Referencias
- https://www.geeksforgeeks.org/random-forest-algorithm-in-machine-learning/ [09/05/2025]